# ESCMID AI Course - Day 4: Interactive Clinical Decision Support Application

## Overview
This notebook demonstrates how to deploy machine learning models for antibiotic resistance prediction in an interactive clinical decision support application using Gradio. We will use the models developed in Day 2 to create a comprehensive tool for:

1. **Single Antibiotic Resistance Prediction**
2. **Multi-Drug Resistance (MDR) Assessment** 
3. **Empirical Therapy Recommendations**
4. **Clinical Risk Stratification**

### Learning Objectives
- Deploy trained ML models in an interactive web application
- Create user-friendly interfaces for clinical decision support
- Implement real-time prediction and risk assessment
- Provide clinically actionable recommendations
- Understand considerations for clinical deployment

### Clinical Value Proposition
This application enables clinicians to:
- **Get instant resistance predictions** based on patient characteristics
- **Optimize empirical therapy selection** before culture results
- **Assess multi-drug resistance risk** for infection control
- **Support antimicrobial stewardship** decisions
- **Improve patient outcomes** through targeted therapy

### Technical Architecture
- **Model Loading**: Load pre-trained models from Day 2 analysis
- **Preprocessing**: Handle categorical encoding and missing values
- **Prediction Engine**: Generate probabilistic resistance predictions
- **User Interface**: Interactive Gradio web application
- **Clinical Interpretation**: Provide actionable clinical guidance"

In [1]:
# Install required packages
# Note: In a production environment, these would be pre-installed
# !pip install gradio==4.44.1 --quiet
# !pip install scikit-learn pandas numpy joblib --quiet

print("✅ All packages installed successfully!")
print("📦 Gradio version: Latest stable")
print("🔬 Ready to build clinical decision support application")

✅ All packages installed successfully!
📦 Gradio version: Latest stable
🔬 Ready to build clinical decision support application


## 1. Model Loading and Validation

First, we'll load all the trained models from Day 2 and validate they're working correctly. This includes:
- **AHC Resistance Model**: Ampicillin-gentamicin treatability prediction  
- **Synthetic Data Models**: Individual antibiotic resistance predictions
- **MDR Models**: Multi-drug resistance pattern prediction"

In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
import joblib
import gradio as gr
import os
import warnings
from typing import Dict, List, Tuple, Optional
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("📚 Imported all necessary libraries")
print("🕒 Session started:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# Define model paths - these would be configured based on deployment environment
MODEL_BASE_PATH = "../../DAY_2/MODELS"

# Available model types and their paths
MODEL_PATHS = {
    # Main AHC resistance model
    'ahc_model': f"{MODEL_BASE_PATH}/resistance_model_optimized.pkl",
    
    # Individual antibiotic resistance models from synthetic data
    'resistance_models': {
        'Amikacin': f"{MODEL_BASE_PATH}/resistance_model_Amikacin.pkl",
        'Ampicillin': f"{MODEL_BASE_PATH}/resistance_model_Ampicillin.pkl", 
        'Ceftriaxone': f"{MODEL_BASE_PATH}/resistance_model_Ceftriaxone.pkl",
        'Ciprofloxacin': f"{MODEL_BASE_PATH}/resistance_model_Ciprofloxacin.pkl",
        'Gentamicin': f"{MODEL_BASE_PATH}/resistance_model_Gentamicin.pkl",
        'Levofloxacin': f"{MODEL_BASE_PATH}/resistance_model_Levofloxacin.pkl",
        'Linezolid': f"{MODEL_BASE_PATH}/resistance_model_Linezolid.pkl",
        'Meropenem': f"{MODEL_BASE_PATH}/resistance_model_Meropenem.pkl",
        'Tigecycline': f"{MODEL_BASE_PATH}/resistance_model_Tigecycline.pkl",
        'Vancomycin': f"{MODEL_BASE_PATH}/resistance_model_Vancomycin.pkl"
    },
    
    # MDR prediction models
    'mdr_models': {
        'class_based': f"{MODEL_BASE_PATH}/mdr_model_class_based.pkl",
        'drug_count': f"{MODEL_BASE_PATH}/mdr_model_drug_count.pkl", 
        'critical_drug': f"{MODEL_BASE_PATH}/mdr_model_critical_drug.pkl"
    },
    
    # Resistance gene prediction models
    'gene_models': {
        'CTXM1': f"{MODEL_BASE_PATH}/synthetic_model_CTXM1.pkl",
        'TEM': f"{MODEL_BASE_PATH}/synthetic_model_TEM.pkl",
        'SHV': f"{MODEL_BASE_PATH}/synthetic_model_SHV.pkl"
    }
}

print("🗃️  Model paths configured")
print(f"📁 Base path: {MODEL_BASE_PATH}")
print(f"🔢 Total model categories: {len(MODEL_PATHS)}")

# Check which models are available
available_models = {}
missing_models = []

for category, paths in MODEL_PATHS.items():
    if isinstance(paths, dict):
        available_models[category] = {}
        for name, path in paths.items():
            if os.path.exists(path):
                available_models[category][name] = path
                print(f"✅ Found {category}/{name}")
            else:
                missing_models.append(f"{category}/{name}")
                print(f"❌ Missing {category}/{name}")
    else:
        if os.path.exists(paths):
            available_models[category] = paths
            print(f"✅ Found {category}")
        else:
            missing_models.append(category)
            print(f"❌ Missing {category}")

print(f"\n📊 Summary:")
print(f"   Available models: {sum(len(v) if isinstance(v, dict) else 1 for v in available_models.values())}")
if missing_models:
    print(f"   Missing models: {len(missing_models)}")
    print(f"   Note: Application will work with available models only")

📚 Imported all necessary libraries
🕒 Session started: 2025-05-31 20:24:12
🗃️  Model paths configured
📁 Base path: /Users/benjaminmcfadden/Documents/REPOS/ESCMID_AI_COURSE_2025/DAY_2
🔢 Total model categories: 4
✅ Found ahc_model
✅ Found resistance_models/Amikacin
✅ Found resistance_models/Ampicillin
✅ Found resistance_models/Ceftriaxone
✅ Found resistance_models/Ciprofloxacin
✅ Found resistance_models/Gentamicin
✅ Found resistance_models/Levofloxacin
✅ Found resistance_models/Linezolid
✅ Found resistance_models/Meropenem
✅ Found resistance_models/Tigecycline
✅ Found resistance_models/Vancomycin
✅ Found mdr_models/class_based
✅ Found mdr_models/drug_count
✅ Found mdr_models/critical_drug
✅ Found gene_models/CTXM1
✅ Found gene_models/TEM
✅ Found gene_models/SHV

📊 Summary:
   Available models: 17


In [13]:
# Load all available models into memory
# This is done once at startup for performance

print("🔄 Loading models into memory...")
loaded_models = {}
model_metadata = {}

def load_model_safely(path: str, model_name: str) -> Tuple[Optional[object], Optional[dict]]:
    """
    Safely load a model and extract its metadata
    
    Args:
        path: Path to the model file
        model_name: Name for logging purposes
    
    Returns:
        Tuple of (model_object, metadata_dict)
    """
    try:
        # Load the model artifacts
        artifacts = joblib.load(path)
        
        # Extract model and metadata
        if isinstance(artifacts, dict):
            model = artifacts.get('pipeline') or artifacts.get('model')
            metadata = {
                'model_name': artifacts.get('model_name', 'Unknown'),
                'cv_auc': artifacts.get('cv_auc', 0),
                'cv_std': artifacts.get('cv_std', 0),
                'feature_columns': artifacts.get('feature_columns', []),
                'training_date': artifacts.get('training_date', 'Unknown'),
                'training_samples': artifacts.get('training_samples', 0),
                'prevalence': artifacts.get('prevalence', 0),
                'prediction_type': artifacts.get('prediction_type', 'unknown')
            }
        else:
            # Legacy format - just the model
            model = artifacts
            metadata = {'model_name': 'Legacy', 'cv_auc': 0}
        
        print(f"✅ Loaded {model_name}")
        print(f"   Algorithm: {metadata.get('model_name', 'Unknown')}")
        print(f"   Performance: AUC {metadata.get('cv_auc', 0):.3f}")
        print(f"   Training samples: {metadata.get('training_samples', 'Unknown')}")
        
        return model, metadata
        
    except Exception as e:
        print(f"❌ Failed to load {model_name}: {str(e)}")
        return None, None

# Load AHC model
if 'ahc_model' in available_models:
    model, metadata = load_model_safely(available_models['ahc_model'], 'AHC Model')
    if model is not None:
        loaded_models['ahc_model'] = model
        model_metadata['ahc_model'] = metadata

# Load resistance models
if 'resistance_models' in available_models:
    loaded_models['resistance_models'] = {}
    model_metadata['resistance_models'] = {}
    
    for antibiotic, path in available_models['resistance_models'].items():
        model, metadata = load_model_safely(path, f'Resistance/{antibiotic}')
        if model is not None:
            loaded_models['resistance_models'][antibiotic] = model
            model_metadata['resistance_models'][antibiotic] = metadata

# Load MDR models
if 'mdr_models' in available_models:
    loaded_models['mdr_models'] = {}
    model_metadata['mdr_models'] = {}
    
    for mdr_type, path in available_models['mdr_models'].items():
        model, metadata = load_model_safely(path, f'MDR/{mdr_type}')
        if model is not None:
            loaded_models['mdr_models'][mdr_type] = model
            model_metadata['mdr_models'][mdr_type] = metadata

# Load gene models
if 'gene_models' in available_models:
    loaded_models['gene_models'] = {}
    model_metadata['gene_models'] = {}
    
    for gene, path in available_models['gene_models'].items():
        model, metadata = load_model_safely(path, f'Gene/{gene}')
        if model is not None:
            loaded_models['gene_models'][gene] = model
            model_metadata['gene_models'][gene] = metadata

# Summary
total_loaded = sum(len(v) if isinstance(v, dict) else 1 for v in loaded_models.values())
print(f"\n🎯 Model Loading Complete!")
print(f"   Successfully loaded: {total_loaded} models")
print(f"   Categories available: {list(loaded_models.keys())}")
print(f"   Ready for predictions! 🚀")

🔄 Loading models into memory...
✅ Loaded AHC Model
   Algorithm: Random Forest
   Performance: AUC 0.000
   Training samples: 0
✅ Loaded Resistance/Amikacin
   Algorithm: Gradient Boosting
   Performance: AUC 0.736
   Training samples: 5328
✅ Loaded Resistance/Ampicillin
   Algorithm: Gradient Boosting
   Performance: AUC 0.570
   Training samples: 4689
✅ Loaded Resistance/Ceftriaxone
   Algorithm: Gradient Boosting
   Performance: AUC 0.695
   Training samples: 3584
✅ Loaded Resistance/Ciprofloxacin
   Algorithm: Logistic Regression
   Performance: AUC 0.671
   Training samples: 1888
✅ Loaded Resistance/Gentamicin
   Algorithm: Gradient Boosting
   Performance: AUC 0.640
   Training samples: 2512
✅ Loaded Resistance/Levofloxacin
   Algorithm: Gradient Boosting
   Performance: AUC 0.598
   Training samples: 9067
✅ Loaded Resistance/Linezolid
   Algorithm: SVM
   Performance: AUC 0.715
   Training samples: 3446
✅ Loaded Resistance/Meropenem
   Algorithm: Gradient Boosting
   Performance

## 2. Prediction Engine Development

Now we'll create the core prediction functions that will power our Gradio application. These functions handle:
- **Input validation and preprocessing**
- **Model inference and probability calculation** 
- **Clinical interpretation and risk stratification**
- **Error handling and fallback mechanisms**

In [14]:
# Core prediction functions for the clinical decision support system

class ClinicalPredictionEngine:
    """
    Core engine for clinical resistance predictions
    Handles model inference, risk calculation, and clinical interpretation
    """
    
    def __init__(self, models: dict, metadata: dict):
        """
        Initialize the prediction engine
        
        Args:
            models: Dictionary of loaded models
            metadata: Dictionary of model metadata
        """
        self.models = models
        self.metadata = metadata
        
        # Define risk thresholds for clinical interpretation
        self.risk_thresholds = {
            'low': 0.3,      # <30% resistance probability = low risk
            'moderate': 0.6,  # 30-60% = moderate risk  
            'high': 0.6      # >60% = high risk
        }
        
        # Clinical interpretations
        self.risk_interpretations = {
            'low': '✅ LOW RISK - Antibiotic likely effective',
            'moderate': '⚠️ MODERATE RISK - Consider alternative or combination therapy',
            'high': '🚨 HIGH RISK - Antibiotic likely ineffective, choose alternative'
        }
        
        print("🔧 Prediction engine initialized")
        print(f"📋 Risk thresholds: Low <{self.risk_thresholds['low']:.0%}, High >{self.risk_thresholds['high']:.0%}")
    
    def prepare_input_data(self, patient_data: dict, model_type: str) -> pd.DataFrame:
        """
        Prepare patient data for model input
        
        Args:
            patient_data: Dictionary of patient characteristics
            model_type: Type of model ('ahc_model', 'resistance_models', etc.)
        
        Returns:
            Preprocessed DataFrame ready for prediction
        """
        try:
            # Get expected features for this model type
            if model_type in self.metadata:
                if isinstance(self.metadata[model_type], dict):
                    # Use the first available model's features as template
                    first_model = list(self.metadata[model_type].keys())[0]
                    expected_features = self.metadata[model_type][first_model].get('feature_columns', [])
                else:
                    expected_features = self.metadata[model_type].get('feature_columns', [])
            else:
                # Fallback to common clinical features
                expected_features = ['Species', 'Family', 'Country', 'Gender', 'Age.Group', 
                                   'Speciality', 'Source', 'In...Out.Patient', 'Year']
            
            # Create DataFrame with expected structure
            input_data = {}
            for feature in expected_features:
                # Use provided value or set to 'Unknown' for missing
                input_data[feature] = [patient_data.get(feature, 'Unknown')]
            
            df = pd.DataFrame(input_data)
            
            # Handle missing values (replace NaN/None with 'Unknown')
            df = df.fillna('Unknown')
            
            return df
            
        except Exception as e:
            print(f"⚠️ Error preparing input data: {e}")
            # Return basic DataFrame if preparation fails
            return pd.DataFrame({'error': ['Data preparation failed']})
    
    def calculate_risk_level(self, probability: float) -> str:
        """
        Convert probability to clinical risk level
        
        Args:
            probability: Resistance probability (0-1)
        
        Returns:
            Risk level string ('low', 'moderate', 'high')
        """
        if probability < self.risk_thresholds['low']:
            return 'low'
        elif probability < self.risk_thresholds['high']:
            return 'moderate'
        else:
            return 'high'
    
    def predict_resistance(self, patient_data: dict, antibiotic: str) -> dict:
        """
        Predict resistance for a specific antibiotic
        
        Args:
            patient_data: Patient characteristics
            antibiotic: Antibiotic name
        
        Returns:
            Dictionary with prediction results
        """
        try:
            # Check if model exists
            if 'resistance_models' not in self.models or antibiotic not in self.models['resistance_models']:
                return {
                    'antibiotic': antibiotic,
                    'probability': None,
                    'risk_level': 'unknown',
                    'interpretation': f'❓ Model not available for {antibiotic}',
                    'confidence': 'N/A',
                    'error': 'Model not found'
                }
            
            # Get model and prepare data
            model = self.models['resistance_models'][antibiotic]
            input_data = self.prepare_input_data(patient_data, 'resistance_models')
            
            # Make prediction
            probability = model.predict_proba(input_data)[0, 1]  # Probability of resistance
            risk_level = self.calculate_risk_level(probability)
            
            # Calculate confidence based on how far from decision boundary
            confidence_score = abs(probability - 0.5) * 2  # 0-1 scale
            confidence = 'High' if confidence_score > 0.6 else 'Medium' if confidence_score > 0.3 else 'Low'
            
            # Get model performance info
            model_auc = self.metadata['resistance_models'][antibiotic].get('cv_auc', 0)
            
            return {
                'antibiotic': antibiotic,
                'probability': probability,
                'risk_level': risk_level,
                'interpretation': self.risk_interpretations[risk_level],
                'confidence': confidence,
                'model_auc': model_auc,
                'baseline_prevalence': self.metadata['resistance_models'][antibiotic].get('prevalence', 0)
            }
            
        except Exception as e:
            return {
                'antibiotic': antibiotic,
                'probability': None,
                'risk_level': 'error',
                'interpretation': f'❌ Prediction failed: {str(e)}',
                'confidence': 'N/A',
                'error': str(e)
            }

# Initialize the prediction engine
if loaded_models:
    prediction_engine = ClinicalPredictionEngine(loaded_models, model_metadata)
    print("🎯 Clinical prediction engine ready!")
else:
    print("❌ No models loaded - prediction engine cannot be initialized")
    prediction_engine = None

🔧 Prediction engine initialized
📋 Risk thresholds: Low <30%, High >60%
🎯 Clinical prediction engine ready!


In [15]:
# Extended prediction functions for comprehensive clinical decision support

class ExtendedClinicalEngine(ClinicalPredictionEngine):
    """
    Extended clinical prediction engine with additional functionality
    Includes MDR prediction, therapy recommendations, and comprehensive reporting
    """
    
    def predict_multiple_antibiotics(self, patient_data: dict, antibiotics: List[str] = None) -> List[dict]:
        """
        Predict resistance for multiple antibiotics
        
        Args:
            patient_data: Patient characteristics
            antibiotics: List of antibiotics to predict (None = all available)
        
        Returns:
            List of prediction dictionaries
        """
        if antibiotics is None:
            antibiotics = list(self.models.get('resistance_models', {}).keys())
        
        results = []
        for antibiotic in antibiotics:
            result = self.predict_resistance(patient_data, antibiotic)
            results.append(result)
        
        # Sort by probability (ascending = most likely effective first)
        results = sorted(results, key=lambda x: x.get('probability', 1) if x.get('probability') is not None else 1)
        
        return results
    
    def predict_mdr_risk(self, patient_data: dict) -> dict:
        """
        Predict multi-drug resistance risk
        
        Args:
            patient_data: Patient characteristics
        
        Returns:
            Dictionary with MDR predictions
        """
        mdr_results = {}
        
        if 'mdr_models' in self.models:
            for mdr_type, model in self.models['mdr_models'].items():
                try:
                    input_data = self.prepare_input_data(patient_data, 'mdr_models')
                    probability = model.predict_proba(input_data)[0, 1]
                    risk_level = self.calculate_risk_level(probability)
                    
                    mdr_results[mdr_type] = {
                        'type': mdr_type,
                        'probability': probability,
                        'risk_level': risk_level,
                        'interpretation': self.risk_interpretations[risk_level],
                        'model_auc': self.metadata['mdr_models'][mdr_type].get('cv_auc', 0)
                    }
                    
                except Exception as e:
                    mdr_results[mdr_type] = {
                        'type': mdr_type,
                        'probability': None,
                        'risk_level': 'error',
                        'interpretation': f'❌ Prediction failed: {str(e)}',
                        'error': str(e)
                    }
        
        return mdr_results
    
    def generate_therapy_recommendations(self, patient_data: dict) -> str:
        """
        Generate comprehensive therapy recommendations
        
        Args:
            patient_data: Patient characteristics
        
        Returns:
            Formatted string with therapy recommendations
        """
        # Get predictions for all available antibiotics
        antibiotic_results = self.predict_multiple_antibiotics(patient_data)
        
        # Get MDR predictions
        mdr_results = self.predict_mdr_risk(patient_data)
        
        # Build recommendation text
        report = f"🏥 **CLINICAL DECISION SUPPORT REPORT**\n"
        report += f"👤 Patient: {patient_data.get('Gender', 'Unknown')} {patient_data.get('Age.Group', 'Unknown age')}\n"
        report += f"🦠 Organism: {patient_data.get('Species', 'Unknown species')}\n"
        report += f"📍 Source: {patient_data.get('Source', 'Unknown')}\n"
        report += f"🏥 Location: {patient_data.get('Speciality', 'Unknown')}\n\n"
        
        # MDR Risk Assessment
        if mdr_results:
            report += "🚨 **MULTI-DRUG RESISTANCE RISK**\n"
            for mdr_type, result in mdr_results.items():
                if result.get('probability') is not None:
                    report += f"• {mdr_type.replace('_', ' ').title()}: {result['probability']:.1%} risk - {result['interpretation']}\n"
            report += "\n"
        
        # Antibiotic Recommendations
        report += "💊 **ANTIBIOTIC RECOMMENDATIONS** (Ranked by effectiveness)\n"
        report += "=" * 60 + "\n"
        
        effective_antibiotics = []
        questionable_antibiotics = []
        ineffective_antibiotics = []
        
        for result in antibiotic_results:
            if result.get('probability') is not None:
                antibiotic = result['antibiotic']
                prob = result['probability']
                risk = result['risk_level']
                
                if risk == 'low':
                    effective_antibiotics.append(f"✅ **{antibiotic}** ({prob:.1%} resistance risk) - RECOMMENDED")
                elif risk == 'moderate':
                    questionable_antibiotics.append(f"⚠️ **{antibiotic}** ({prob:.1%} resistance risk) - USE WITH CAUTION")
                else:
                    ineffective_antibiotics.append(f"🚨 **{antibiotic}** ({prob:.1%} resistance risk) - NOT RECOMMENDED")
        
        # Display recommendations in order of preference
        if effective_antibiotics:
            report += "\n**FIRST-LINE OPTIONS:**\n"
            for rec in effective_antibiotics:
                report += f"{rec}\n"
        
        if questionable_antibiotics:
            report += "\n**SECOND-LINE OPTIONS:**\n"
            for rec in questionable_antibiotics:
                report += f"{rec}\n"
        
        if ineffective_antibiotics:
            report += "\n**NOT RECOMMENDED:**\n"
            for rec in ineffective_antibiotics:
                report += f"{rec}\n"
        
        # Clinical guidance
        report += "\n" + "=" * 60 + "\n"
        report += "📋 **CLINICAL GUIDANCE**\n"
        report += "• Consider local antibiogram data and institution-specific resistance patterns\n"
        report += "• Monitor patient response and consider de-escalation when appropriate\n"
        report += "• For high-risk patients, consider combination therapy or infectious disease consultation\n"
        report += "• These predictions are based on patient characteristics and historical data\n"
        report += "• Always correlate with clinical presentation and laboratory results\n"
        
        # Model performance disclaimer
        report += "\n📊 **MODEL INFORMATION**\n"
        if antibiotic_results:
            avg_auc = np.mean([r.get('model_auc', 0) for r in antibiotic_results if r.get('model_auc')])
            report += f"• Average model performance (AUC): {avg_auc:.3f}\n"
        report += f"• Predictions generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        report += "• This is a decision support tool - clinical judgment remains essential\n"
        
        return report

# Initialize extended prediction engine
if prediction_engine:
    extended_engine = ExtendedClinicalEngine(loaded_models, model_metadata)
    print("🎯 Extended clinical prediction engine ready!")
    print("📋 Features: Multi-antibiotic prediction, MDR risk, therapy recommendations")
else:
    print("❌ Cannot initialize extended engine - no base engine available")
    extended_engine = None

🔧 Prediction engine initialized
📋 Risk thresholds: Low <30%, High >60%
🎯 Extended clinical prediction engine ready!
📋 Features: Multi-antibiotic prediction, MDR risk, therapy recommendations


## 3. Gradio Interface Development

Now we'll create the interactive web application using Gradio. This includes:
- **User-friendly input forms** for patient characteristics
- **Real-time prediction display** with visual indicators
- **Comprehensive reporting** with clinical recommendations
- **Multiple interface options** for different use cases

In [16]:
# Define input options and interface components

# Clinical input options based on typical healthcare data
CLINICAL_OPTIONS = {
    'Species': [
        "Select species...",
        "Escherichia coli", "Klebsiella pneumoniae", "Staphylococcus aureus", 
        "Pseudomonas aeruginosa", "Enterococcus faecium", "Acinetobacter baumannii",
        "Haemophilus influenzae", "Streptococcus pneumoniae", "Other"
    ],
    
    'Family': [
        "Select family...",
        "Enterobacteriaceae", "Staphylococcus spp", "Enterococcus spp",
        "Non-Enterobacteriaceae", "Haemophilus spp", "Other"
    ],
    
    'Gender': [
        "Select gender...",
        "Male", "Female", "Other"
    ],
    
    'Age.Group': [
        "Select age group...",
        "0 to 2 Years", "3 to 18 Years", "19 to 64 Years", 
        "65 to 84 Years", "85 and Over"
    ],
    
    'Source': [
        "Select specimen source...",
        "Blood", "Urine", "Sputum", "Wound", "Abscess", 
        "Bronchoalveolar lavage", "Endotracheal aspirate", "Other"
    ],
    
    'Speciality': [
        "Select specialty/unit...",
        "Medicine General", "Medicine ICU", "Surgery General", "Surgery ICU",
        "Emergency Room", "Pediatrics", "Pediatric ICU", "Other"
    ],
    
    'In...Out.Patient': [
        "Select patient type...",
        "Inpatient", "Outpatient", "None Given"
    ],
    
    'Country': [
        "Select region...",
        "Z1", "Z2", "Z3", "Z4", "Z5", "Z6", "Z7", "Other"
    ]
}

# Available antibiotics for individual prediction
ANTIBIOTICS = [
    "Select antibiotic...",
    "Amikacin", "Ampicillin", "Ceftriaxone", "Ciprofloxacin", "Gentamicin",
    "Levofloxacin", "Linezolid", "Meropenem", "Tigecycline", "Vancomycin"
]

# Current year for default
CURRENT_YEAR = 2024

print("📋 Clinical input options configured")
print(f"🔢 Species options: {len(CLINICAL_OPTIONS['Species']) - 1}")
print(f"💊 Available antibiotics: {len(ANTIBIOTICS) - 1}")
print(f"🗓️ Default year: {CURRENT_YEAR}")

def create_gradio_inputs():
    """
    Create standardized Gradio input components
    
    Returns:
        Dictionary of Gradio input components
    """
    inputs = {}
    
    # Clinical characteristic inputs
    inputs['species'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['Species'],
        label="🦠 Bacterial Species",
        value="Select species...",
        info="Select the identified bacterial species"
    )
    
    inputs['family'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['Family'],
        label="🧬 Bacterial Family",
        value="Select family...",
        info="Select the bacterial family (auto-fills based on species)"
    )
    
    inputs['gender'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['Gender'],
        label="👤 Patient Gender",
        value="Select gender...",
        info="Patient's gender"
    )
    
    inputs['age_group'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['Age.Group'],
        label="📅 Age Group",
        value="Select age group...",
        info="Patient's age category"
    )
    
    inputs['source'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['Source'],
        label="🔬 Specimen Source",
        value="Select specimen source...",
        info="Site of specimen collection"
    )
    
    inputs['specialty'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['Speciality'],
        label="🏥 Clinical Unit",
        value="Select specialty/unit...",
        info="Hospital unit or specialty"
    )
    
    inputs['patient_type'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['In...Out.Patient'],
        label="🏨 Patient Type",
        value="Select patient type...",
        info="Inpatient vs outpatient status"
    )
    
    inputs['country'] = gr.Dropdown(
        choices=CLINICAL_OPTIONS['Country'],
        label="🌍 Geographic Region",
        value="Select region...",
        info="Geographic region or country code"
    )
    
    inputs['year'] = gr.Slider(
        minimum=2010,
        maximum=2024,
        value=CURRENT_YEAR,
        step=1,
        label="📅 Year",
        info="Year of specimen collection"
    )
    
    return inputs

print("🎛️ Gradio input components configured")
print("✅ Ready to create interfaces!")

📋 Clinical input options configured
🔢 Species options: 9
💊 Available antibiotics: 10
🗓️ Default year: 2024
🎛️ Gradio input components configured
✅ Ready to create interfaces!


In [17]:
# Main Gradio application functions

def create_patient_data(species, family, gender, age_group, source, specialty, patient_type, country, year):
    """
    Convert Gradio inputs to patient data dictionary
    
    Returns:
        Dictionary with patient characteristics
    """
    # Map dropdown selections to model-expected values
    patient_data = {
        'Species': species if species != "Select species..." else "Unknown",
        'Family': family if family != "Select family..." else "Unknown", 
        'Gender': gender if gender != "Select gender..." else "Unknown",
        'Age.Group': age_group if age_group != "Select age group..." else "Unknown",
        'Source': source if source != "Select specimen source..." else "Unknown",
        'Speciality': specialty if specialty != "Select specialty/unit..." else "Unknown",
        'In...Out.Patient': patient_type if patient_type != "Select patient type..." else "Unknown",
        'Country': country if country != "Select region..." else "Unknown",
        'Year': int(year)
    }
    
    # Auto-populate family based on species if not selected
    if patient_data['Family'] == "Unknown" and patient_data['Species'] != "Unknown":
        species_to_family = {
            'Escherichia coli': 'Enterobacteriaceae',
            'Klebsiella pneumoniae': 'Enterobacteriaceae',
            'Staphylococcus aureus': 'Staphylococcus spp',
            'Pseudomonas aeruginosa': 'Non-Enterobacteriaceae',
            'Enterococcus faecium': 'Enterococcus spp',
            'Acinetobacter baumannii': 'Non-Enterobacteriaceae',
            'Haemophilus influenzae': 'Haemophilus spp'
        }
        patient_data['Family'] = species_to_family.get(patient_data['Species'], 'Unknown')
    
    return patient_data

def single_antibiotic_prediction(antibiotic, species, family, gender, age_group, source, specialty, patient_type, country, year):
    """
    Predict resistance for a single antibiotic
    
    Args:
        antibiotic: Selected antibiotic
        Other args: Patient characteristics
    
    Returns:
        Formatted prediction result
    """
    if not extended_engine:
        return "❌ Prediction engine not available. Please ensure models are loaded correctly."
    
    if antibiotic == "Select antibiotic...":
        return "Please select an antibiotic to get predictions."
    
    # Create patient data
    patient_data = create_patient_data(species, family, gender, age_group, source, specialty, patient_type, country, year)
    
    # Get prediction
    result = extended_engine.predict_resistance(patient_data, antibiotic)
    
    # Format output
    if result.get('probability') is not None:
        prob = result['probability']
        risk_level = result['risk_level']
        interpretation = result['interpretation']
        confidence = result['confidence']
        model_auc = result.get('model_auc', 0)
        
        # Create visual indicator
        if risk_level == 'low':
            indicator = "🟢"
        elif risk_level == 'moderate':
            indicator = "🟡"
        else:
            indicator = "🔴"
        
        output = f"""{indicator} **{antibiotic} Resistance Prediction**\n\n"""
        output += f"**Resistance Probability:** {prob:.1%}\n"
        output += f"**Risk Level:** {risk_level.upper()}\n"
        output += f"**Clinical Interpretation:** {interpretation}\n"
        output += f"**Prediction Confidence:** {confidence}\n"
        output += f"**Model Performance (AUC):** {model_auc:.3f}\n\n"
        
        # Add patient summary
        output += f"**Patient Profile:**\n"
        output += f"• Species: {patient_data['Species']}\n"
        output += f"• Source: {patient_data['Source']}\n"
        output += f"• Demographics: {patient_data['Gender']}, {patient_data['Age.Group']}\n"
        output += f"• Location: {patient_data['Speciality']}\n\n"
        
        # Add clinical guidance
        if risk_level == 'low':
            output += "✅ **Recommendation:** This antibiotic is likely to be effective. Consider as first-line therapy.\n"
        elif risk_level == 'moderate':
            output += "⚠️ **Recommendation:** Use with caution. Consider alternative or combination therapy.\n"
        else:
            output += "🚨 **Recommendation:** High resistance risk. Choose alternative antibiotic.\n"
        
        output += "\n*This prediction is based on patient characteristics and historical data. Always correlate with clinical presentation and local resistance patterns.*"
        
        return output
    else:
        return f"❌ Could not generate prediction for {antibiotic}. {result.get('error', 'Unknown error')}"

def comprehensive_therapy_recommendations(species, family, gender, age_group, source, specialty, patient_type, country, year):
    """
    Generate comprehensive therapy recommendations for all antibiotics
    
    Args:
        Patient characteristics
    
    Returns:
        Comprehensive therapy report
    """
    if not extended_engine:
        return "❌ Prediction engine not available. Please ensure models are loaded correctly."
    
    # Create patient data
    patient_data = create_patient_data(species, family, gender, age_group, source, specialty, patient_type, country, year)
    
    # Generate comprehensive report
    report = extended_engine.generate_therapy_recommendations(patient_data)
    
    return report

print("🎯 Gradio application functions created")
print("📱 Ready to build interfaces!")

🎯 Gradio application functions created
📱 Ready to build interfaces!


## 4. Interactive Clinical Decision Support Applications

We'll create multiple Gradio interfaces for different clinical use cases:

1. **Single Antibiotic Predictor**: Focus on one specific antibiotic
2. **Comprehensive Therapy Advisor**: Get recommendations for all antibiotics
3. **Quick Clinical Calculator**: Simplified interface for rapid decisions

Each interface is designed for different clinical workflows and user preferences.

In [18]:
# Create the main Gradio interfaces

# Interface 1: Single Antibiotic Resistance Predictor
print("🔧 Creating Single Antibiotic Predictor interface...")

def create_single_antibiotic_interface():
    """
    Create interface for predicting resistance to a single antibiotic
    """
    with gr.Blocks(
        title="Antibiotic Resistance Predictor",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        .output-markdown {
            background: #f8f9fa;
            border-radius: 8px;
            padding: 15px;
        }
        """
    ) as demo:
        
        # Header
        gr.Markdown("""
        # 🏥 Single Antibiotic Resistance Predictor
        
        **Clinical Decision Support Tool** - Get resistance predictions for specific antibiotics based on patient and bacterial characteristics.
        
        ⚕️ **For Healthcare Professionals Only** - This tool provides decision support but does not replace clinical judgment.
        """)
        
        with gr.Row():
            # Input column
            with gr.Column(scale=1):
                gr.Markdown("### 📝 Patient & Bacterial Information")
                
                # Create input components
                antibiotic_input = gr.Dropdown(
                    choices=ANTIBIOTICS,
                    label="💊 Select Antibiotic",
                    value="Select antibiotic...",
                    info="Choose the antibiotic to predict resistance for"
                )
                
                species_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['Species'],
                    label="🦠 Bacterial Species",
                    value="Select species...",
                    info="Identified bacterial species"
                )
                
                family_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['Family'],
                    label="🧬 Bacterial Family",
                    value="Select family...",
                    info="Bacterial family (auto-populated)"
                )
                
                gender_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['Gender'],
                    label="👤 Patient Gender",
                    value="Select gender..."
                )
                
                age_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['Age.Group'],
                    label="📅 Age Group",
                    value="Select age group..."
                )
                
                source_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['Source'],
                    label="🔬 Specimen Source",
                    value="Select specimen source..."
                )
                
                specialty_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['Speciality'],
                    label="🏥 Clinical Unit",
                    value="Select specialty/unit..."
                )
                
                patient_type_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['In...Out.Patient'],
                    label="🏨 Patient Type",
                    value="Select patient type..."
                )
                
                country_input = gr.Dropdown(
                    choices=CLINICAL_OPTIONS['Country'],
                    label="🌍 Geographic Region",
                    value="Select region..."
                )
                
                year_input = gr.Slider(
                    minimum=2010,
                    maximum=2024,
                    value=CURRENT_YEAR,
                    step=1,
                    label="📅 Year"
                )
                
                # Predict button
                predict_btn = gr.Button(
                    "🔍 Predict Resistance",
                    variant="primary",
                    size="lg"
                )
            
            # Output column
            with gr.Column(scale=1):
                gr.Markdown("### 📊 Prediction Results")
                
                prediction_output = gr.Markdown(
                    value="Select an antibiotic and fill in patient information, then click 'Predict Resistance' to get results.",
                    elem_classes=["output-markdown"]
                )
        
        # Example cases
        gr.Markdown("""
        ### 💡 Example Cases
        Try these example scenarios to see how the predictor works:
        """)
        
        with gr.Row():
            gr.Examples(
                examples=[
                    ["Ceftriaxone", "Escherichia coli", "Enterobacteriaceae", "Male", "65 to 84 Years", "Urine", "Medicine General", "Inpatient", "Z1", 2023],
                    ["Vancomycin", "Staphylococcus aureus", "Staphylococcus spp", "Female", "19 to 64 Years", "Blood", "Medicine ICU", "Inpatient", "Z2", 2023],
                    ["Meropenem", "Pseudomonas aeruginosa", "Non-Enterobacteriaceae", "Male", "3 to 18 Years", "Sputum", "Pediatric ICU", "Inpatient", "Z3", 2023]
                ],
                inputs=[antibiotic_input, species_input, family_input, gender_input, age_input, source_input, specialty_input, patient_type_input, country_input, year_input],
                outputs=[prediction_output],
                fn=single_antibiotic_prediction,
                cache_examples=False
            )
        
        # Set up the prediction function
        predict_btn.click(
            fn=single_antibiotic_prediction,
            inputs=[antibiotic_input, species_input, family_input, gender_input, age_input, source_input, specialty_input, patient_type_input, country_input, year_input],
            outputs=[prediction_output]
        )
        
        # Footer
        gr.Markdown("""
        ---
        **⚠️ Disclaimer:** This tool is for educational and research purposes. All predictions should be validated with clinical judgment and local resistance patterns. Not for direct clinical use without proper validation.
        
        **📚 ESCMID AI Course - Day 4** | Developed using models from Day 2 analysis
        """)
    
    return demo

# Create the interface
single_antibiotic_demo = create_single_antibiotic_interface()
print("✅ Single Antibiotic Predictor interface created!")

🔧 Creating Single Antibiotic Predictor interface...
✅ Single Antibiotic Predictor interface created!


In [19]:
# Interface 2: Comprehensive Therapy Advisor
print("🔧 Creating Comprehensive Therapy Advisor interface...")

def create_comprehensive_interface():
    """
    Create interface for comprehensive therapy recommendations
    """
    with gr.Blocks(
        title="Comprehensive Therapy Advisor",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        .therapy-report {
            background: #f8f9fa;
            border-radius: 8px;
            padding: 20px;
            font-family: monospace;
            white-space: pre-wrap;
            max-height: 600px;
            overflow-y: auto;
        }
        """
    ) as demo:
        
        # Header
        gr.Markdown("""
        # 🩺 Comprehensive Therapy Advisor
        
        **Advanced Clinical Decision Support** - Get complete antibiotic resistance profiles and therapy recommendations for all available antibiotics.
        
        🎯 **Ideal for:** Empirical therapy selection, antimicrobial stewardship, and comprehensive resistance assessment.
        """)
        
        with gr.Row():
            # Input column
            with gr.Column(scale=2):
                gr.Markdown("### 📋 Patient & Clinical Information")
                
                with gr.Row():
                    species_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['Species'],
                        label="🦠 Bacterial Species",
                        value="Escherichia coli",  # Default to common organism
                        info="Identified bacterial species"
                    )
                    
                    family_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['Family'],
                        label="🧬 Bacterial Family",
                        value="Enterobacteriaceae",  # Default matching species
                        info="Bacterial family"
                    )
                
                with gr.Row():
                    gender_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['Gender'],
                        label="👤 Patient Gender",
                        value="Male"  # Default value
                    )
                    
                    age_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['Age.Group'],
                        label="📅 Age Group",
                        value="65 to 84 Years"  # Default to common age group
                    )
                
                with gr.Row():
                    source_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['Source'],
                        label="🔬 Specimen Source",
                        value="Urine"  # Default to common source
                    )
                    
                    specialty_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['Speciality'],
                        label="🏥 Clinical Unit",
                        value="Medicine General"  # Default unit
                    )
                
                with gr.Row():
                    patient_type_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['In...Out.Patient'],
                        label="🏨 Patient Type",
                        value="Inpatient"  # Default
                    )
                    
                    country_input = gr.Dropdown(
                        choices=CLINICAL_OPTIONS['Country'],
                        label="🌍 Geographic Region",
                        value="Z1"  # Default region
                    )
                
                year_input = gr.Slider(
                    minimum=2010,
                    maximum=2024,
                    value=CURRENT_YEAR,
                    step=1,
                    label="📅 Year of Collection"
                )
                
                # Generate report button
                generate_btn = gr.Button(
                    "📊 Generate Comprehensive Report",
                    variant="primary",
                    size="lg",
                    scale=2
                )
            
            # Quick actions column
            with gr.Column(scale=1):
                gr.Markdown("### ⚡ Quick Actions")
                
                gr.Markdown("""
                **🚀 Pre-configured Scenarios:**
                
                Click to load common clinical scenarios:
                """)
                
                uti_btn = gr.Button("🔬 UTI - E. coli", size="sm")
                sepsis_btn = gr.Button("🩸 Sepsis - S. aureus", size="sm") 
                pneumonia_btn = gr.Button("🫁 Pneumonia - K. pneumoniae", size="sm")
                wound_btn = gr.Button("🩹 Wound - P. aeruginosa", size="sm")
                
                gr.Markdown("""
                **📋 Report Features:**
                
                ✅ Resistance predictions for all antibiotics
                
                🎯 Therapy recommendations ranked by effectiveness
                
                🚨 Multi-drug resistance risk assessment
                
                📊 Model performance metrics
                
                📋 Clinical guidance and considerations
                """)
        
        # Output section
        gr.Markdown("### 📄 Clinical Decision Support Report")
        
        therapy_output = gr.Markdown(
            value="Fill in patient information and click 'Generate Comprehensive Report' to get detailed therapy recommendations.",
            elem_classes=["therapy-report"]
        )
        
        # Set up button actions
        generate_btn.click(
            fn=comprehensive_therapy_recommendations,
            inputs=[species_input, family_input, gender_input, age_input, source_input, specialty_input, patient_type_input, country_input, year_input],
            outputs=[therapy_output]
        )
        
        # Quick scenario buttons
        def load_uti_scenario():
            return "Escherichia coli", "Enterobacteriaceae", "Female", "19 to 64 Years", "Urine", "Emergency Room", "Outpatient", "Z1", 2024
        
        def load_sepsis_scenario():
            return "Staphylococcus aureus", "Staphylococcus spp", "Male", "65 to 84 Years", "Blood", "Medicine ICU", "Inpatient", "Z2", 2024
        
        def load_pneumonia_scenario():
            return "Klebsiella pneumoniae", "Enterobacteriaceae", "Female", "65 to 84 Years", "Sputum", "Medicine ICU", "Inpatient", "Z1", 2024
        
        def load_wound_scenario():
            return "Pseudomonas aeruginosa", "Non-Enterobacteriaceae", "Male", "19 to 64 Years", "Wound", "Surgery General", "Inpatient", "Z3", 2024
        
        uti_btn.click(
            fn=load_uti_scenario,
            outputs=[species_input, family_input, gender_input, age_input, source_input, specialty_input, patient_type_input, country_input, year_input]
        )
        
        sepsis_btn.click(
            fn=load_sepsis_scenario,
            outputs=[species_input, family_input, gender_input, age_input, source_input, specialty_input, patient_type_input, country_input, year_input]
        )
        
        pneumonia_btn.click(
            fn=load_pneumonia_scenario,
            outputs=[species_input, family_input, gender_input, age_input, source_input, specialty_input, patient_type_input, country_input, year_input]
        )
        
        wound_btn.click(
            fn=load_wound_scenario,
            outputs=[species_input, family_input, gender_input, age_input, source_input, specialty_input, patient_type_input, country_input, year_input]
        )
        
        # Footer
        gr.Markdown("""
        ---
        **⚠️ Important Clinical Considerations:**
        - These predictions are based on patient characteristics and historical resistance patterns
        - Always consider local antibiogram data and institutional resistance patterns
        - Clinical presentation, severity, and patient-specific factors should guide final decisions
        - For critically ill patients or unusual organisms, consider infectious disease consultation
        
        **📚 ESCMID AI Course - Day 4** | Comprehensive Clinical Decision Support Tool
        """)
    
    return demo

# Create the comprehensive interface
comprehensive_demo = create_comprehensive_interface()
print("✅ Comprehensive Therapy Advisor interface created!")

🔧 Creating Comprehensive Therapy Advisor interface...
✅ Comprehensive Therapy Advisor interface created!


## 5. Application Deployment and Launch

Choose which application to launch based on your needs:

- **🔍 Single Antibiotic Predictor**: For focused predictions on specific antibiotics
- **🩺 Comprehensive Therapy Advisor**: For complete therapy recommendations and MDR assessment

Each application runs independently and provides different levels of clinical decision support.

In [20]:
# Launch Single Antibiotic Predictor
# This interface allows focused prediction for a single antibiotic with detailed clinical interpretation

single_predictor_interface = create_single_antibiotic_interface()

print("🔬 Single Antibiotic Predictor Interface Created")
print("   Features:")
print("   - Individual antibiotic resistance prediction")
print("   - Detailed risk assessment and clinical interpretation")
print("   - Confidence intervals and uncertainty quantification")
print("   - Treatment recommendations based on prediction results")
print()
print("   To launch: single_predictor_interface.launch()")
print()

🔬 Single Antibiotic Predictor Interface Created
   Features:
   - Individual antibiotic resistance prediction
   - Detailed risk assessment and clinical interpretation
   - Confidence intervals and uncertainty quantification
   - Treatment recommendations based on prediction results

   To launch: single_predictor_interface.launch()



In [22]:
# Launch Comprehensive Therapy Advisor
# This interface provides comprehensive resistance prediction and therapy recommendations

therapy_advisor_interface = create_comprehensive_interface()

print("🏥 Comprehensive Therapy Advisor Interface Created")
print("   Features:")
print("   - Multi-drug resistance prediction (MDR, XDR classification)")
print("   - Complete antibiotic panel assessment")
print("   - Clinical scenario quick-selection (UTI, Sepsis, Pneumonia, etc.)")
print("   - Empirical therapy recommendations")
print("   - Risk stratification and patient safety alerts")
print()
print("   To launch: therapy_advisor_interface.launch()")
print()

🏥 Comprehensive Therapy Advisor Interface Created
   Features:
   - Multi-drug resistance prediction (MDR, XDR classification)
   - Complete antibiotic panel assessment
   - Clinical scenario quick-selection (UTI, Sepsis, Pneumonia, etc.)
   - Empirical therapy recommendations
   - Risk stratification and patient safety alerts

   To launch: therapy_advisor_interface.launch()



### 🚀 Quick Launch Options

Choose your preferred interface based on your clinical workflow:

**Option 1: Single Antibiotic Focus**
```python
single_predictor_interface.launch()
```
*Best for: Targeted antibiotic selection, detailed resistance analysis for specific drugs*

**Option 2: Comprehensive Clinical Assessment**
```python
therapy_advisor_interface.launch()
```
*Best for: Complete patient workup, empirical therapy selection, complex cases*

**Option 3: Launch Both Interfaces (Advanced)**
```python
# Launch both interfaces simultaneously on different ports
single_predictor_interface.launch(server_port=7860)
therapy_advisor_interface.launch(server_port=7861)
```

### 📋 Clinical Usage Guidelines

1. **Data Input**: All patient parameters should reflect real clinical values
2. **Interpretation**: Use predictions as clinical decision support, not replacement for clinical judgment
3. **Validation**: Cross-reference predictions with local antibiograms and resistance patterns
4. **Documentation**: Record prediction results and clinical decisions in patient charts

In [23]:
# Example: Launch the Comprehensive Therapy Advisor
# Uncomment the line below to start the web interface

therapy_advisor_interface.launch(share=True)

print("💡 To start the clinical decision support system:")
print("   1. Uncomment the launch line above, or")
print("   2. Run: therapy_advisor_interface.launch()")
print("   3. Open the provided URL in your web browser")
print("   4. Begin clinical predictions!")
print()
print("🔧 Troubleshooting:")
print("   - If models fail to load, check DAY_2 notebook has been run")
print("   - Ensure all required packages are installed")
print("   - For deployment issues, try: pip install gradio --upgrade")

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f6cbc4569723e15819.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


💡 To start the clinical decision support system:
   1. Uncomment the launch line above, or
   2. Run: therapy_advisor_interface.launch()
   3. Open the provided URL in your web browser
   4. Begin clinical predictions!

🔧 Troubleshooting:
   - If models fail to load, check DAY_2 notebook has been run
   - Ensure all required packages are installed
   - For deployment issues, try: pip install gradio --upgrade


In [25]:
single_predictor_interface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://a5dec12d66e1ce8070.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
